In [1]:
import pandas as pd
import seaborn as sns 
import numpy as np
import pickle

In [2]:
df_orders = pd.read_csv("orders_before_dec.csv", sep = "|")
df_items = pd.read_csv("items.csv", sep = "|")
df_category = pd.read_csv("category_hierarchy.csv", sep = "|")

print(len(df_orders), "orders")
print(len(df_items), "items")
print(len(df_category), "categories")

767131 orders
32776 items
4332 categories


In [3]:
data = pd.merge(df_orders, df_items, on="itemID")
print(len(data), "data entries")
data.head()

767131 data entries


,date,userID,itemID,order,brand,feature_1,feature_2,feature_3,feature_4,feature_5,categories
0,2020-06-01,38769,3477,1,186,6,0,196,0,45,"[74, 4109, 3867, 803, 4053]"
1,2020-06-01,23570,3477,1,186,6,0,196,0,45,"[74, 4109, 3867, 803, 4053]"
2,2020-06-01,23495,3477,1,186,6,0,196,0,45,"[74, 4109, 3867, 803, 4053]"
3,2020-06-01,26073,3477,1,186,6,0,196,0,45,"[74, 4109, 3867, 803, 4053]"
4,2020-06-01,34302,3477,1,186,6,0,196,0,45,"[74, 4109, 3867, 803, 4053]"


In [4]:
data = data.sort_values(["userID", "itemID"])
originaldata = data
data = data.copy()
data["last_purchase"] = -1
data["next_purchase"] = -1
# data = data[(data["userID"]>5000) & (data["userID"]<=7500)]
data = data[(data["userID"]<=31)]
data_length = len(data)
data_length

500

In [5]:
is_replen_file = open("is_replen.pickle", "rb")
is_replen = pickle.load(is_replen_file)
is_replen_file.close()

In [6]:
existing_data_file = open("existing_data.pickle", "rb")
existing_data = pickle.load(existing_data_file)
existing_data_file.close()

In [7]:
idx = 0
for i, row in data.iterrows():
    if idx % 10000 == 0:
        print(idx, "of", data_length)
    
    if idx >= data_length:
        print("Done")
        break
    
    # if the item is a replenishment, set the last purchase date
    if idx > 0 and is_replen[idx]==True:
        data.loc[idx, 'last_purchase'] = 1 # data.iloc[idx-1]["date"]
    
    
    if idx+1 < data_length and is_replen[idx+1]==True:
        data.loc[idx, 'next_purchase'] = 1 # data.iloc[idx+1]["date"]
    
    idx+=1

print(len(data))
# data[data["userID"]==31]

0 of 500
616


In [60]:
print("previous data length:", len(existing_data))
new_data = pd.concat([existing_data, data])
print("new data length:", len(new_data))
print(new_data.head())

previous data length: 100564
new data length: 101180
         brand                                         categories        date  \
511233   286.0                  [3392, 179, 1390, 563, 383, 1886]  2020-09-01   
587354   107.0                [3024, 1772, 1136, 3882, 948, 4071]  2020-11-20   
499786  1288.0                                 [3224, 2690, 2700]  2020-08-03   
293416    53.0  [1073, 3529, 3005, 3574, 1760, 3436, 1472, 226...  2020-08-18   
151236   286.0  [3392, 179, 1390, 563, 383, 1886, 3912, 1763, ...  2020-08-03   

        feature_1  feature_2  feature_3  feature_4  feature_5   itemID  \
511233        4.0        0.0       82.0        0.0      144.0   1505.0   
587354        6.0        0.0      308.0        3.0       17.0   9325.0   
499786       10.0        0.0      421.0        3.0        3.0  12468.0   
293416        4.0        3.0      474.0        0.0       -1.0  12505.0   
151236        4.0        0.0       82.0        0.0      144.0  15083.0   

       last_pur

/Users/daniel/opt/anaconda3/envs/pytorchenv/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [48]:
new_data[new_data["userID"]==31]

,date,userID,itemID,order,brand,feature_1,feature_2,feature_3,feature_4,feature_5,categories,last_purchase,next_purchase
482528,2020-08-08,31,1358,1,386,10,0,503,0,85,"[3654, 1760, 268, 3297, 1851, 1578, 1085, 2995...",-1,-1
134764,2020-10-01,31,10376,1,449,4,0,535,3,105,"[2629, 1760, 1730, 471, 3915, 1915, 3914, 609,...",-1,-1
355331,2020-10-01,31,10548,1,1279,6,0,424,3,102,"[2682, 2811, 3915]",-1,-1
682581,2020-08-08,31,13558,1,1201,10,0,386,0,147,"[956, 614]",-1,-1
682584,2020-10-01,31,13558,1,1201,10,0,386,0,147,"[956, 614]",-1,-1
638992,2020-08-08,31,14117,1,1163,10,0,236,0,85,"[3639, 1792, 1696, 3874, 1579]",-1,-1
61804,2020-08-08,31,14214,1,322,4,0,536,0,144,"[3078, 2322, 160]",-1,-1
415831,2020-08-08,31,27919,1,993,4,0,487,3,144,"[3900, 3910, 3905, 3903, 189, 975]",-1,-1
414427,2020-10-11,31,28531,7,1371,10,0,344,0,95,"[2443, 3173, 3245]",-1,-1
588278,2020-08-08,31,30263,1,993,4,0,486,0,144,"[1980, 2873, 3325]",-1,-1


In [28]:
training_data_file = open("existing_data.pickle", "wb")
pickle.dump(new_data, training_data_file)

training_data_file.close()

In [41]:
print(new_data.head())

NameError: name 'new_data' is not defined

## Compute replenishments

In [5]:
is_replenishment = [False for i in range(len(data))]
for i in range(1, len(data)):
    if i % 10000 == 0:
        print(i)
    
    if data.iloc[i]["itemID"] == data.iloc[i-1]["itemID"]:
        is_replenishment[i] = True

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000


In [8]:
is_replen_file = open("is_replen.pickle", "wb")
pickle.dump(is_replenishment, is_replen_file)

is_replen_file.close()